In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!pip install Sastrawi
!pip install translate
!pip install tweet-preprocessor
!pip install textblob
!pip install wordcloud
!pip install nltk
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
import preprocessor as p
from textblob import TextBlob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Data Cleaning

In [ ]:
df = pd.read_csv('muhaimin.csv')
df = df[['full_text', 'username', 'created_at']]
df

,full_text,username,created_at
0,@VIVAcoid Iya betul survei memenangkan PS DAN ...,saaleepp88,Mon Dec 25 10:56:40 +0000 2023
1,@liputan6dotcom Keberagaman ide dan sudut pand...,azzura26052000,Tue Dec 26 15:40:19 +0000 2023
2,Untuk Meningkatkan Pemahaman Agar mudah Sosial...,msw_andi,Mon Dec 25 10:50:09 +0000 2023
3,@CNNIndonesia Gus Muhaimin dan Anies Baswedan ...,azzura26052000,Tue Dec 26 15:39:45 +0000 2023
4,"@ajengcute16__ Bismillah.. Amin, Anies -Muhaim...",ado_ono,Mon Dec 25 10:49:27 +0000 2023
...,...,...,...
395,"@DokterTifa @cakimiNOW G msk akaaalllll, orang...",risnand59179688,Mon Dec 25 02:41:41 +0000 2023
396,"@Heraloebss Bismillah.. Amin, Anies -Muhaimin ...",ado_ono,Mon Dec 25 02:40:45 +0000 2023
397,"@Arie_Kriting Bismillah.. Amin, Anies -Muhaimi...",ado_ono,Mon Dec 25 02:39:01 +0000 2023
398,"@VIVAcoid Bismillah.. Amin, Anies -Muhaimin P...",ado_ono,Mon Dec 25 02:35:11 +0000 2023


In [ ]:
df.duplicated().sum()

0

In [ ]:
df = df.dropna()
df.isnull().sum()

full_text     0
username      0
created_at    0
dtype: int64

In [ ]:
def cleanTwitter(text):
  text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)
  text = re.sub(r'#\w+', ' ', text)
  text = re.sub(r'RT[\s]+', ' ', text)
  text = re.sub(r'https?://\S+', ' ', text)
  text = re.sub(r'[^A-Za-z0-9 ]', ' ', text)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

df['full_text'] = df['full_text'].apply(cleanTwitter)
df['full_text'] = df['full_text'].str.lower()

In [ ]:
df = df.drop_duplicates(subset=['full_text'])

In [ ]:
df

,full_text,username,created_at
0,iya betul survei memenangkan ps dan gibran tet...,saaleepp88,Mon Dec 25 10:56:40 +0000 2023
1,keberagaman ide dan sudut pandang anies baswed...,azzura26052000,Tue Dec 26 15:40:19 +0000 2023
2,untuk meningkatkan pemahaman agar mudah sosial...,msw_andi,Mon Dec 25 10:50:09 +0000 2023
3,gus muhaimin dan anies baswedan telah mengukir...,azzura26052000,Tue Dec 26 15:39:45 +0000 2023
4,bismillah amin anies muhaimin presiden dan wak...,ado_ono,Mon Dec 25 10:49:27 +0000 2023
...,...,...,...
392,calon wakil presiden cawapres nomor urut satu ...,VIVAcoid,Mon Dec 25 02:49:55 +0000 2023
393,sepertnya masih kurang tuh orang mengimami sho...,ado_ono,Mon Dec 25 02:47:39 +0000 2023
394,deklarasi dan pernyatan sikap masyarakat sukab...,Hero212_reborn,Mon Dec 25 02:46:14 +0000 2023
395,g msk akaaalllll orang ditanya sm rakyat sm ma...,risnand59179688,Mon Dec 25 02:41:41 +0000 2023


# Preprocessing

In [ ]:
# Set the display option
# pd.set_option('display.max_colwidth', None)

for i, text in enumerate(df['full_text']):
  print(f'{i}. {text}')

In [ ]:
#normalisasi


norm = {
    "gibran": "",
    "mahfud": "",
    " ri ": " indonesia ",
    " gak ": " tidak ",
    " yg ": " yang ",
    " nggak ": " tidak ",
    " jg ": " juga ",
    " tlh ": " telah ",
    " ttp ": " tetap ",
    " donk ": " ",
    " tp ": " tetapi ",
    " ud ": " sudah ",
    " lu ": " anda ",
    " ga ": " tidak ",
    " dg ": " dengan ",
    " dgn ": " dengan ",
    " sdh ": " sudah ",
    " tdk ": " tidak ",
    " g ": " tidak ",
    "gw": "saya",
    "gua": "saya",
    " tak ": " tidak ",
    " prabowo ": " ",
    " ganjar ": " ",
    "siang": "",
    "selamat natal": "",
    "christmas": "",
    "home alone": "",
    "lembang": "",
    "miskha": "",
    "fuji": "",
    "istj": "",
    "wa aja": "",
    "palestine": "",
    "pukis": "",
    "morowali": "",
    "choi minho": "",
    "drmn": "darimana",
    " kl ": " jikalau ",
    " jd ": " jadi ",
    " kmrn ": " kemarin ",
    " msk ": " masuk ",
    " sm ": " sama ",
    " sj ": " saja "
}


def normalisasi(text):
  for i in norm:
    text = text.replace(i, norm[i])
  return text

df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))
df

,full_text,username,created_at
0,iya betul survei memenangkan ps dan tetap pre...,saaleepp88,Mon Dec 25 10:56:40 +0000 2023
1,keberagaman ide dan sudut pandang anies baswed...,azzura26052000,Tue Dec 26 15:40:19 +0000 2023
2,untuk meningkatkan pemahaman agar mudah sosial...,msw_andi,Mon Dec 25 10:50:09 +0000 2023
3,gus muhaimin dan anies baswedan telah mengukir...,azzura26052000,Tue Dec 26 15:39:45 +0000 2023
4,bismillah amin anies muhaimin presiden dan wak...,ado_ono,Mon Dec 25 10:49:27 +0000 2023
...,...,...,...
392,calon wakil presiden cawapres nomor urut satu ...,VIVAcoid,Mon Dec 25 02:49:55 +0000 2023
393,sepertnya masih kurang tuh orang mengimami sho...,ado_ono,Mon Dec 25 02:47:39 +0000 2023
394,deklarasi dan pernyatan sikap masyarakat sukab...,Hero212_reborn,Mon Dec 25 02:46:14 +0000 2023
395,g masuk akaaalllll orang ditanya sama rakyat s...,risnand59179688,Mon Dec 25 02:41:41 +0000 2023


In [ ]:
#tokenize
tokenized = df['full_text'].apply(lambda x:x.split())
tokenized

0      [iya, betul, survei, memenangkan, ps, dan, tet...
1      [keberagaman, ide, dan, sudut, pandang, anies,...
2      [untuk, meningkatkan, pemahaman, agar, mudah, ...
3      [gus, muhaimin, dan, anies, baswedan, telah, m...
4      [bismillah, amin, anies, muhaimin, presiden, d...
                             ...                        
392    [calon, wakil, presiden, cawapres, nomor, urut...
393    [sepertnya, masih, kurang, tuh, orang, mengima...
394    [deklarasi, dan, pernyatan, sikap, masyarakat,...
395    [g, masuk, akaaalllll, orang, ditanya, sama, r...
399                             [masih, anies, muhaimin]
Name: full_text, Length: 346, dtype: object

In [ ]:
#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in text:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  return d_clean

tokenized = tokenized.apply(stemming)
tokenized.to_csv("preProcessingMuhaimin.csv", index=False)

In [ ]:
data = pd.read_csv('preProcessingMuhaimin.csv')